In [11]:
# read ans process files 

import pandas as pd
import numpy as np
import json
import csv
import re





In [12]:
df = pd.read_csv('iot-light.tsv',sep='\t', na_values=['null','None','',], dtype=object)
df.fillna('')
df.head(5)


,TweetID,Sentiment,TopicID,Country,Gender,URLs,Created_at,Text,Source,in_reply_to_status_id_str,...,User_id,User_name,User_location,User_followers_count,Coordinates,quoted_status_id_str,is_quote_status,Entities_hashtags,Entities_urls,LDA_Topic
0,1099404026,neutral,NaN,ca,male,https://en.wikipedia.org/wiki/Internet_of_Things,Tue Jan 06 13:22:32 +0000 2009,The Internet of Things. L'internet des objets....,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",NaN,...,14350762,Jacques Cool,"Montréal, Québec",10126,NaN,NaN,False,False,NaN,-1
1,1099881858,neutral,2,gb,andy,NaN,Tue Jan 06 17:20:36 +0000 2009,programming mirroir and nabaztag... Internet o...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",NaN,...,12318782,alezante,"London, England",1641,NaN,NaN,False,False,NaN,-1
2,1104012014,neutral,1,be,male,NaN,Thu Jan 08 09:08:26 +0000 2009,aha mycrocosm integration & internet of things...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",NaN,...,16113118,Dries De Roeck,"Antwerp, Belgium",1298,NaN,NaN,False,False,NaN,-1
3,1104556075,neutral,0,it,andy,NaN,Thu Jan 08 15:34:16 +0000 2009,The concept is basically The Internet of Things,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",NaN,...,18074191,Zeugmamedia,"Florence, Italy",87,NaN,NaN,False,False,NaN,-1
4,1104557021,neutral,2,it,andy,NaN,Thu Jan 08 15:34:51 +0000 2009,As I see the evolution: Internet of Websites -...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",NaN,...,18074191,Zeugmamedia,"Florence, Italy",87,NaN,NaN,False,False,NaN,-1


In [13]:
df['LDA_Topic'].unique()

array(['-1', '4', '5', '0', '3'], dtype=object)

# Users

In [14]:
topic_x = df[df['LDA_Topic']=='5'] ;

In [15]:
topic_x.groupby('Gender').count()['TweetID']

Gender
andy           1
mostly_male    1
Name: TweetID, dtype: int64

In [16]:
#topic_x[topic_x['Gender']=='male'][['Country', 'User_location', 'Coordinates']]
topic_x[topic_x['Gender']=='male']['Country']

Series([], Name: Country, dtype: object)

In [ ]:
df.columns

Tableau

In [18]:
topic_0 = df[df['LDA_Topic']=='0'] ;
topic_1 = df[df['LDA_Topic']=='1'] ;
topic_2 = df[df['LDA_Topic']=='2'] ;
topic_3 = df[df['LDA_Topic']=='3'] ;
topic_4 = df[df['LDA_Topic']=='4'] ;
topic_5 = df[df['LDA_Topic']=='5'] ;

Nb tweets par topics

In [21]:
print(topic_0.count()['TweetID'])
print(topic_1.count()['TweetID'])
print(topic_2.count()['TweetID'])
print(topic_3.count()['TweetID'])
print(topic_4.count()['TweetID'])
print(topic_5.count()['TweetID'])

1
0
0
1
26
2


nb reponse par topics

In [57]:
nbr_0 = 0
for index,row in topic_0.iterrows():
    a = str(row['in_reply_to_status_id_str'])
    if len(a) > 5:
        nbr_0 = nbr_0 + 1
print (nbr_0)

nbr_1 = 0
for index,row in topic_1.iterrows():
    a = str(row['in_reply_to_status_id_str'])
    if len(a) > 5:
        nbr_1 = nbr_1 + 1
print (nbr_1)

nbr_2 = 0
for index,row in topic_2.iterrows():
    a = str(row['in_reply_to_status_id_str'])
    if len(a) > 5:
        nbr_2 = nbr_2 + 1
print (nbr_2)

nbr_3 = 0
for index,row in topic_3.iterrows():
    a = str(row['in_reply_to_status_id_str'])
    if len(a) > 5:
        nbr_3 = nbr_3 + 1
print (nbr_3)

nbr_4= 0
for index,row in topic_4.iterrows():
    a = str(row['in_reply_to_status_id_str'])
    if len(a) > 5:
        nbr_4 = nbr_4 + 1
print (nbr_4)

nbr_5= 0
for index,row in topic_5.iterrows():
    a = str(row['in_reply_to_status_id_str'])
    if len(a) > 5:
        nbr_5 = nbr_5 + 1
print (nbr_5)


1
0
0
1
23
1


nb tweets d'influencers

In [59]:
nbi_0 = 0
for index,row in topic_0.iterrows():
    print(row['User_followers_count'])
    if int(row['User_followers_count']) > 1000:
        nbi_0 = nbi_0 + 1
print (nbi_0)

227
0


# Sentiment

In [43]:
#topic_x = df[df['LDA_Topic']=='5'] ;
topic_x.groupby('Sentiment').count()['TweetID']

Sentiment
negative      410
neutral     16382
positive     1352
Name: TweetID, dtype: int64

# Hashtags

In [63]:
#topic_x = df[df['LDA_Topic']=='5'] ;
dict_ht = {}
total_ht = []
for index,row in topic_x.iterrows():
    #l_ht = []
    for word in row['Text'].split():
        if word[0] == '#':
            #l_ht.append(word)
            total_ht.append(word.lower())
    #print(row['Text'].split())

#Creating an empty dictionary  
freq = {} 
for item in total_ht: 
    if (item in freq): 
        freq[item] += 1
    else: 
        freq[item] = 1

sorted_x = sorted(freq.items(), key=lambda kv: kv[1], reverse=True)
#[ x[0] for x in sorted_x[0:7] ]
[ x[1] for x in sorted_x[0:7] ]

[14089, 1289, 998, 625, 451, 401, 314]

# Time series

In [26]:
from datetime import datetime

In [66]:
dictM = {
    'Jan' : '01',
    'Feb' : '02',
    'Mar' : '03',
    'Apr' : '04',
    'May' : '05',
    'Jun' : '06',
    'Jul' : '07',
    'Aug' : '08',
    'Sep' : '09',
    'Oct' : '10',
    'Nov' : '11',
    'Dec' : '12',
}

In [105]:
#topic_x = df[df['LDA_Topic']=='5'] ;
tfreq = {}
for row in topic_x['Created_at'] :
    dt = row.split()
    #t = dt[2]+' '+dt[1]+' '+dt[-1]
    t = (dictM[dt[1]],dt[-1])
    if t in tfreq :
        tfreq[t] += 1
    else:
        tfreq[t] = 1
    #print(t)
    #t = datetime.strptime(row, '%d %m %y ')
    # dd MMM yyyy HH:mm:ss
    #t = datetime.fromtimestamp(row)
    #print(t)

In [ ]:
#tfreq
#[ [x[0],x[1]] for x in tfreq ]
[ tfreq[x] for x in tfreq ]

## Users time series

In [139]:
topic_x = df[df['LDA_Topic']=='5'] ;
    
tfreq = {}
for a,row in topic_x[['Created_at', 'User_id']].iterrows() :
    #print(row)
    dt = row['Created_at'].split()
    un = row['User_id']
    #t = dt[2]+' '+dt[1]+' '+dt[-1]
    t = dt[1]+' '+dt[-1]
    if t in tfreq :
        if not row['User_id'] in tfreq[t] :
            tfreq[t].append(un)
    else:
        tfreq[t] = [un]

In [140]:
print("[")
for k,v in tfreq.items() :
    dd= k.split()
    #print([dd[0], len(v)])
    #print([dictM[dd[0]],dd[1]], ",")
    print(len(v),",")
print("]")

[
2 ,
2 ,
3 ,
4 ,
3 ,
1 ,
1 ,
1 ,
3 ,
2 ,
4 ,
1 ,
3 ,
1 ,
2 ,
8 ,
3 ,
1 ,
7 ,
2 ,
2 ,
8 ,
6 ,
2 ,
1 ,
10 ,
6 ,
7 ,
11 ,
11 ,
6 ,
8 ,
7 ,
16 ,
15 ,
4 ,
11 ,
15 ,
13 ,
14 ,
21 ,
12 ,
32 ,
40 ,
65 ,
59 ,
71 ,
58 ,
56 ,
120 ,
132 ,
114 ,
137 ,
231 ,
175 ,
241 ,
262 ,
267 ,
294 ,
277 ,
318 ,
314 ,
399 ,
330 ,
384 ,
471 ,
526 ,
635 ,
514 ,
430 ,
455 ,
376 ,
369 ,
354 ,
352 ,
282 ,
240 ,
348 ,
355 ,
343 ,
340 ,
327 ,
305 ,
283 ,
288 ,
290 ,
308 ,
269 ,
231 ,
]


#  Sample Text

In [150]:
pd.set_option('max_colwidth', 800)
for x in range(0,6):
    print("Topic ",x)
    topic_x = df[df['LDA_Topic']=='5']
    print(topic_x.sample(n=15)['Text'])

Topic  0
6248077                           The 50 Most Influential Women in #IoT  https://t.co/6uQzvJnQfU … … #womenintech  @evankirstel https://t.co/0sXhARl0EU
2672519                                          @Things_Internet bye bye wired connectivity way to represent #IoT #wireless activity #wirelesscharging
4796057    @nanoview @adamajacoby @FinancialReview let's add #AI and #IoT .Oh and the census data which is now compulsory ($100 fine/day) and public...
2388681           .@SamHetchler Thanks for the link Sam! All you #IoT devs out there check out http://t.co/IRqmKEDPjg including https://t.co/fkuzCvXy3B
1277948                     "Innovation in the #IoT space has been happening all around the world. #Europe has been very active" http://t.co/4okv3AgZpK
3281829          .@AutomationGT Robo-Prius carpools and robo-deliveries could fight traffic congestion and reduce the need for parking #IoT #automation
1388330              @joannaonthelake INTERESTING, TNX! stay up2date #IoT #Wear

5893560             RT @AndreaSDPiazza Understading #customerexperience journey is key to leverage #IoT and data collected from touchpoints #CX #HBRwebinar
5558511        @Street_Insider #BuyDips Stay #LoNG #FB #Facebook is a BeAST! #BEASTMODE #247365 #Algorithm #IOT #Connect #World #iG https://t.co/65vrSxCToU
611888     MT .@M2MWORLDNEWS #TELIT ILS &amp; Jasper Collaborate to help #Mobile Operators Deliver Innovative #IoT Applications #M2M http://t.co/MjDDj9tWOH
3697250          Why @salesforce and #IoT? Trusted, Secure, Agile, Open API’s, Partner approach ISV’s and SI's, Proven - Continuous innovation and upgrades
3225835                “@securityaffairs: #Hacking Baby Monitors is dramatically easynhttps://t.co/WuSV10HnFyn#securityaffairs #BabyMonitors #IoT” #InfoSec
3067683                                        @SCMatSAP Have a look into our new Cloud Trial for SAP Networked Logistics Hub #IoT - http://t.co/qocggIr04H
802449                                                          